<a href="https://colab.research.google.com/github/SukhmanpreetKaurManes/Langchain-Project/blob/main/News_Summerizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install langchain openai requests python-dotenv langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 3.1 MB/s eta 0:00:00


In [ ]:
import os

# Paste your OpenRouter API key and NewsAPI key here
os.environ["OPENROUTER_API_KEY"] = "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"
NEWS_API_KEY = "ccdbd0d714584efdb7639422c7c60c1c"


In [38]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
#ChatOpenAI: Initializes your LLM interface with the API key
#PromptTemplate: Lets you create reusable prompt formats with variables you can swap in dynamically.
#LLMChain: Chains the prompt with the model and takes care of executing the pipeline

llm = ChatOpenAI(
    openai_api_base="https://openrouter.ai/api/v1",
    openai_api_key=os.environ["OPENROUTER_API_KEY"],
    model="mistralai/mistral-small-3.1-24b-instruct:free",
    temperature=0.5,    #  the temperature=0.5 parameter controls the randomness or creativity of the model’s output.
    max_tokens=150
)


In [13]:
def setup_summarizer(llm):
    from langchain.prompts import PromptTemplate

    prompt = PromptTemplate(
        input_variables=["article"],
        template="Summarize the following news article in simple language:\n\n{article}"
    )

    chain = prompt | llm    #wiring your prompt directly to the model.
    return chain



In [23]:
import requests

def get_top_news(api_key, country="us", category="general", count=3):
    url = f"https://newsapi.org/v2/top-headlines?country={country}&category={category}&apiKey={api_key}"
    response = requests.get(url)
    articles = response.json().get("articles", [])[:count]
    return articles

#response.json() Converts the raw response from an HTTP request (via requests.get) into a Python dictionary.

#.get("articles", []) Retrieves the value associated with the "articles" key. If that key doesn’t exist, it returns an empty list by default.

#[:count] Slices the list to return only the first count items.


In [14]:
def summarize_articles(articles, summarizer):
    summaries = []
    for i, article in enumerate(articles):
        content = article.get("description") or article.get("content") or ""
        if not content.strip():
            continue
        result = summarizer.invoke({"article": content})
        summaries.append((article["title"], result))
    return summaries


In [39]:
# Step 1: Fetch news
articles = get_top_news(NEWS_API_KEY)
print(f"Number of articles fetched: {len(articles)}")
for i, a in enumerate(articles, 1):
    print(f"\n{i}. Title: {a.get('title')}")
    print(f"   Description: {a.get('description')}")

# Step 2: Setup summarizer
summarizer = setup_summarizer(llm)

# Step 3: Get and print summaries
summaries = summarize_articles(articles, summarizer)

# Display
for i, (title, summary) in enumerate(summaries, 1):
    print(f"\n📰 Article {i}: {title}\n📄 Summary: {summary}\n{'-'*80}")


Number of articles fetched: 3

1. Title: Weekly News Quiz - AARP
   Description: How much do you know about AARP’s big stories?

2. Title: Column | Israeli plans for Gaza draw criticism of ‘concentration camps’ - The Washington Post
   Description: Israeli Defense Minister Israel Katz has instructed the military to prepare to build a “humanitarian city” in Gaza that critics describe as a “concentration camp.”

3. Title: 3I/Atlas: Mystery interstellar object could be the oldest known comet - BBC
   Description: Scientists have been racing to discover the origins of 3I/Atlas since it was spotted last week.

📰 Article 1: Weekly News Quiz - AARP
📄 Summary: content='Sure, here\'s a simple summary of the topic:\n\nAARP, which stands for American Association of Retired Persons, is an organization that focuses on issues important to people aged 50 and over. They cover various topics that are significant to their members, such as healthcare, retirement planning, social security, and other conce